In [1]:
from preprocessing import *
from sklearn.model_selection import KFold
import argparse
from model import *
from train import test
import torch.optim as optim
import pandas as pd

from MatrixVectorizer import *
import networkx as nx
from typing import Union


In [2]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data])

num_samples = hr_train_data_vectorized.shape[0]
num_samples_list = range(num_samples)
sample_to_index = dict(zip(num_samples_list, hr_train_data_vectorized))

split = int(num_samples * 0.8)

In [3]:

subjects_adj,subjects_labels = lr_train_data_vectorized[:split], hr_train_data_vectorized[:split]

held_out_subjects_adj,held_out_subjects_labels = lr_train_data_vectorized[split:], hr_train_data_vectorized[split:]

In [4]:
num_splt = 3
epochs = 50
lr = 0.00005 # try [0.0001, 0.0005, 0.00001, 0.00005]
lmbda = 17 # should be around 15-20
lamdba_topo = 1 # should be around 0.5-1.5
lr_dim = 160
hr_dim = 320
hidden_dim = 320 # try smaller and larger - [160-512]
padding = 26
dropout = 0.1 # try [0., 0.1, 0.2, 0.3]
args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [5]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [6]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [7]:
# precompute topological measures for hr_train_data_vectorized
# precomputed_measures = precompute_topological_measures(hr_train_data_vectorized)

In [8]:
import time
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj,subjects_labels, args): 
  #, subjects_adj_test, subjects_ground_truth_test):
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []
    epoch_topo = []

    model.train()
    for index,(lr,hr) in enumerate(zip(subjects_adj,subjects_labels)):      
      lr = torch.from_numpy(lr).type(torch.FloatTensor)
      hr = torch.from_numpy(hr).type(torch.FloatTensor)

      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs  = unpad(model_outputs, args.padding)

      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
      start_time = time.time()

      topo = args.lamdba_topo * compute_topological_MAE_loss(hr, model_outputs)

      error = criterion(model_outputs, hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      epoch_topo.append(topo.item())
      
  
    model.eval()
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error),
          "Topo: ", np.mean(epoch_topo))
    all_epochs_loss.append(np.mean(epoch_loss))

In [9]:
# # print(model)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
# # optimizer = optim.SGD(model.parameters(), lr=args.lr)

# for train_index, test_index in cv.split(subjects_adj):
#     subjects_adj_train = subjects_adj[train_index]  # Get training data 
#     subjects_adj_test = subjects_adj[test_index]   # Get testing data 
#     subjects_ground_truth_train = subjects_labels[train_index]
#     subjects_ground_truth_test = subjects_labels[test_index]

#     train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args, subjects_adj_test, subjects_ground_truth_test)
    
#     print('Held out test score:')
#     test(model, held_out_subjects_adj, held_out_subjects_labels, args)
#     print('------------------------------')

# Final Model & Kaggle Submission

In [10]:
#final train
final_model = GSRNet(ks, args)
optimizer = optim.Adam(final_model.parameters(), lr=args.lr)

train(final_model, optimizer, lr_train_data_vectorized, hr_train_data_vectorized, args)

Epoch:  1 Loss:  5.750217312110398 Error:  0.24174252429051313 Topo:  56.494220973488815
Epoch:  2 Loss:  4.644323122001694 Error:  0.21155612670375915 Topo:  38.04522014092542
Epoch:  3 Loss:  4.265881241438631 Error:  0.2021683175228313 Topo:  32.09554512914783
Epoch:  4 Loss:  4.006610543428067 Error:  0.19598122869066137 Topo:  28.440001990266904
Epoch:  5 Loss:  3.8055684595050927 Error:  0.1935964972673062 Topo:  27.180524888866675
Epoch:  6 Loss:  3.6367760960927265 Error:  0.19193236494135713 Topo:  26.311932106931767
Epoch:  7 Loss:  3.488994397089153 Error:  0.1904636861499912 Topo:  25.598259189171706
Epoch:  8 Loss:  3.3557335745074792 Error:  0.1891832515865029 Topo:  25.002406971183365
Epoch:  9 Loss:  3.2329834506897157 Error:  0.18738922565997004 Topo:  24.30307106771869
Epoch:  10 Loss:  3.118971824645996 Error:  0.186167123431931 Topo:  23.79939566972013
Epoch:  11 Loss:  3.011815520817648 Error:  0.18534923019166477 Topo:  23.46538637355416
Epoch:  12 Loss:  2.909726

In [11]:
#Generate submission 

# load csvs as numpy
test_lr_data_path = '../data/lr_test.csv'

# lr_test_data = np.loadtxt(test_lr_data_path, delimiter=',')
lr_test_data = pd.read_csv(test_lr_data_path, delimiter=',').to_numpy()
print(lr_test_data.shape)
lr_test_data[lr_test_data < 0] = 0
np.nan_to_num(lr_test_data, copy=False)


# map the anti-vectorize function to each row of the lr_train_data

lr_test_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_test_data])
print(lr_test_data_vectorized.shape)

(112, 12720)
(112, 160, 160)


In [12]:
final_model.eval()
preds = []
for lr in lr_test_data_vectorized:      
  lr = torch.from_numpy(lr).type(torch.FloatTensor)
  
  model_outputs, _, _, _ = final_model(lr)
  model_outputs  = unpad(model_outputs, args.padding)
  preds.append(MatrixVectorizer.vectorize(model_outputs.detach().numpy()))

print(len(preds), preds[0].shape)
r = np.hstack(preds)
print(r.shape)
meltedDF = r.flatten()

112 (35778,)
(4007136,)


In [13]:
n = meltedDF.shape[0]
df = pd.DataFrame({'ID': np.arange(1, n+1),
                   'Predicted': meltedDF})
df.to_csv('submission.csv', index=False)